Union 2.1 Dataset Analysis
==========================

This data uses the distance modulus, $\mu$, to describe the distance rather than the flux or apparent magnitude. We have
$$
    \mu = m - M~,
$$
where $M$ is given by the dataset as being $M(h=0.7) = -19.3081547178$ and is defined below.

So the dataset gives $\mu(z)$ so we need an extra step.

We will be again fitting the six-parameter $\Lambda$CDM model.

In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg', 'retina'}

In [7]:
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import lmfit

In [8]:
hubble_constant = 70 * 3.24078e-20
speed_of_light = 3e8

In [9]:
def a_of_z(z):
    """Simple function that gives a(z) = 1/(1+z)"""
    return 1./(1+z)

In [10]:
def h_of_a(a, omega_c=0, omega_b=0.3, omega_rad=0, omega_k=0, omega_de=0.7, w=-1, H0=hubble_constant):
    """Implementation of (1) at the top. Returns H(a)"""
    return H0 * np.sqrt((omega_c+omega_b)*(a**(-3)) + omega_rad*(a**(-4)) + omega_k*(a**(-2)) + omega_de*(a**(-3*(1+w))))

In [11]:
def h_of_z(z, omega_c=0, omega_b=0.3, omega_rad=0, omega_k=0, omega_de=0.7, w=-1, H0=hubble_constant):
    """Implementation of (1) at the top. Returns H(a)"""
    a = a_of_z(z)
    return h_of_a(a, omega_c, omega_b, omega_rad, omega_k, omega_de, w, H0)

In [12]:
def h_of_z_inverse(z, omega_c=0, omega_b=0.3, omega_rad=0, omega_k=0, omega_de=0.7, w=-1, H0=hubble_constant):
    """1/h_of_z"""
    return 1/h_of_z(z, omega_c, omega_b, omega_rad, omega_k, omega_de, w, H0)

In [13]:
def f_of_z(z, lpeak, r_eta):
    """Implementation of (2) at the top. Returns flux as a function of redshift"""
    return lpeak/(4*np.pi*((r_eta)**2)*((1+z)**2))

$$
    m = m_0 - 2.5\log_{10}(f)
$$
$$
    M - m = mu

In [ ]:
def mu_of_f(f, m0=-19.3):
    """Gives distance modulus as a function of f"""
    return 